In [ ]:
import numpy as np 

N = 3
M = 1

c_0 = np.ones(N) / np.sqrt(N) 

F = np.ones(N) 
L = 2/N * np.ones((N,N))

for i in np.arange(N):
    if i >= N-M:
        F[i]*= -1
for i in np.arange(N):
    for j in np.arange(N):
        delta=0
        if i==j:
            delta=1
        L[i,j] = (2/N - delta)*F[j]





array([[-0.00411523,  0.99588477,  0.09053498],
       [ 0.99588477, -0.00411523,  0.09053498],
       [-0.09053498, -0.09053498,  0.99176955]])

In [19]:
N = 10 
M = 3 

u = np.sqrt(N-M) / np.sqrt(N) * (1 - 2*M* (N+1)/N**2)
m = np.sqrt(M) / np.sqrt(N) * (1 + 2/N - 2*M* (N+1)/N**2)

np.sqrt(M) / np.sqrt(N)

np.float64(0.5477225575051661)